# 国能日新 光伏功率预测

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import datetime
import time
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectPercentile,f_regression
pd.options.display.max_columns = None  # 显示的最大列数不受限制

In [2]:
train_1 = pd.read_csv('./data2/train/train_1.csv')
train_2 = pd.read_csv('./data2/train/train_2.csv')
train_3 = pd.read_csv('./data2/train/train_3.csv')
train_4 = pd.read_csv('./data2/train/train_4.csv')
train_5 = pd.read_csv('./data2/train/train_5.csv')
train_6 = pd.read_csv('./data2/train/train_6.csv')
train_7 = pd.read_csv('./data2/train/train_7.csv')
train_8 = pd.read_csv('./data2/train/train_8.csv')
train_9 = pd.read_csv('./data2/train/train_9.csv')
train_10 = pd.read_csv('./data2/train/train_10.csv')

test_1 = pd.read_csv('./data2/test/test_1.csv')
test_2 = pd.read_csv('./data2/test/test_2.csv')
test_3 = pd.read_csv('./data2/test/test_3.csv')
test_4 = pd.read_csv('./data2/test/test_4.csv')
test_5 = pd.read_csv('./data2/test/test_5.csv')
test_6 = pd.read_csv('./data2/test/test_6.csv')
test_7 = pd.read_csv('./data2/test/test_7.csv')
test_8 = pd.read_csv('./data2/test/test_8.csv')
test_9 = pd.read_csv('./data2/test/test_9.csv')
test_10 = pd.read_csv('./data2/test/test_10.csv')

In [9]:
data_1 = pd.concat([train_1,test_1],axis=0,ignore_index=True,sort=False)
data_2 = pd.concat([train_2,test_2],axis=0,ignore_index=True,sort=False)
data_3 = pd.concat([train_3,test_3],axis=0,ignore_index=True,sort=False)
data_4 = pd.concat([train_4,test_4],axis=0,ignore_index=True,sort=False)
data_5 = pd.concat([train_5,test_5],axis=0,ignore_index=True,sort=False)
data_6 = pd.concat([train_6,test_6],axis=0,ignore_index=True,sort=False)
data_7 = pd.concat([train_7,test_7],axis=0,ignore_index=True,sort=False)
data_8 = pd.concat([train_8,test_8],axis=0,ignore_index=True,sort=False)
data_9 = pd.concat([train_9,test_9],axis=0,ignore_index=True,sort=False)
data_10 = pd.concat([train_10,test_10],axis=0,ignore_index=True,sort=False)

In [10]:
# f1 = pd.DataFrame(data_4.groupby("风速")["风向"].count()).reset_index().rename(columns={"风向":"count of 风速 and 风向"})
# f2 = pd.DataFrame(data_4.groupby("风速")["风向"].mean()).reset_index().rename(columns={"风向":"mean of 风速 and 风向"})
# f3 = pd.DataFrame(data_4.groupby("风速")["风向"].nunique()).reset_index().rename(columns={"风向":"nuq of 风速 and 风向"})
# f4 = pd.DataFrame(data_4.groupby("湿度")["风向"].mean()).reset_index().rename(columns={"风向":"mean of 湿度 and 风向"})
# f5 = pd.DataFrame(data_4.groupby("风向")["风速"].mean()).reset_index().rename(columns={"风速":"mean of 风向 and 风速"})

In [11]:
# data_4 = pd.merge(data_4,f1, how='left', on=["风速"])
# data_4 = pd.merge(data_4,f2, how='left', on=["风速"])
# data_4 = pd.merge(data_4,f3, how='left', on=["风速"])
# data_4 = pd.merge(data_4,f4, how='left', on=["湿度"])
# data_4 = pd.merge(data_4,f5, how='left', on=["风向"])

In [12]:
# # 返回时间戳
# def get_timestamp(x):
#     try:
#         t = datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S.%f")
#         t = int(round((t.timestamp())))  # 四舍五入，去掉毫秒级  
#     except:
#         t = datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S")
#         t = int(t.timestamp())
#     return t

In [13]:
data_1.head()

,时间,辐照度,风速,风向,温度,湿度,压强,实际辐照度,实际功率,id
0,2017-01-01 00:15:00,-1.0,-0.855915,0.291529,-0.459561,-0.207037,0.043513,0.0,0.039000,NaN
1,2017-01-01 00:30:00,-1.0,-0.851950,0.395271,-0.462905,-0.200263,0.042722,0.0,0.039000,NaN
2,2017-01-01 00:45:00,-1.0,-0.847984,0.497236,-0.465831,-0.192790,0.041930,0.0,0.042333,NaN
3,2017-01-01 01:00:00,-1.0,-0.844019,0.592254,-0.469592,-0.184763,0.041139,0.0,0.042667,NaN
4,2017-01-01 01:15:00,-1.0,-0.838731,0.675103,-0.473354,-0.176346,0.040348,0.0,0.042667,NaN


In [15]:
# 数据类型
data_1.dtypes

时间        object
辐照度      float64
风速       float64
风向       float64
温度       float64
湿度       float64
压强       float64
实际辐照度    float64
实际功率     float64
id       float64
dtype: object

In [16]:
data_1["时间戳"] = pd.to_datetime(data_1["时间"])
data = data_1[['时间戳','辐照度','风速','风向','温度','压强','湿度']]
data = data.drop_duplicates(["时间戳"]) # 去掉重复值，不去掉合并会多行

In [17]:
t1 = data.copy()

In [18]:
t1.head()

,时间戳,辐照度,风速,风向,温度,压强,湿度
0,2017-01-01 00:15:00,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037
1,2017-01-01 00:30:00,-1.0,-0.851950,0.395271,-0.462905,0.042722,-0.200263
2,2017-01-01 00:45:00,-1.0,-0.847984,0.497236,-0.465831,0.041930,-0.192790
3,2017-01-01 01:00:00,-1.0,-0.844019,0.592254,-0.469592,0.041139,-0.184763
4,2017-01-01 01:15:00,-1.0,-0.838731,0.675103,-0.473354,0.040348,-0.176346


### 借位 + 时间，当前气象特征和多久前的气象特征合并，第一行不变，增加特征

In [19]:
t1.columns = ['时间戳','辐照度-1','风速-1','风向-1','温度-1','压强-1','湿度-1']

In [20]:
# 获取第一行，用于合并
t1_0 = t1[:1].copy()

In [21]:
# 以15分钟为例
t1["时间戳"] = t1["时间戳"].map(lambda x: x + pd.Timedelta(seconds=900))
t1 = pd.concat([t1_0, t1], axis=0)
data_1 = pd.merge(data_1, t1, how="left", on="时间戳")

In [22]:
data_1.head()

,时间,辐照度,风速,风向,温度,湿度,压强,实际辐照度,实际功率,id,时间戳,辐照度-1,风速-1,风向-1,温度-1,压强-1,湿度-1
0,2017-01-01 00:15:00,-1.0,-0.855915,0.291529,-0.459561,-0.207037,0.043513,0.0,0.039000,NaN,2017-01-01 00:15:00,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037
1,2017-01-01 00:30:00,-1.0,-0.851950,0.395271,-0.462905,-0.200263,0.042722,0.0,0.039000,NaN,2017-01-01 00:30:00,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037
2,2017-01-01 00:45:00,-1.0,-0.847984,0.497236,-0.465831,-0.192790,0.041930,0.0,0.042333,NaN,2017-01-01 00:45:00,-1.0,-0.851950,0.395271,-0.462905,0.042722,-0.200263
3,2017-01-01 01:00:00,-1.0,-0.844019,0.592254,-0.469592,-0.184763,0.041139,0.0,0.042667,NaN,2017-01-01 01:00:00,-1.0,-0.847984,0.497236,-0.465831,0.041930,-0.192790
4,2017-01-01 01:15:00,-1.0,-0.838731,0.675103,-0.473354,-0.176346,0.040348,0.0,0.042667,NaN,2017-01-01 01:15:00,-1.0,-0.844019,0.592254,-0.469592,0.041139,-0.184763


In [23]:
t2 = data.copy()
t2.columns = ['时间戳','辐照度-2','风速-2','风向-2','温度-2','压强-2','湿度-2']
t2_0 = t2[:2].copy() # 获取第一行，用于合并
t2['时间戳'] = t2['时间戳'].map(lambda x: x+pd.Timedelta(seconds=1800))  #30分*60=1800秒
t2 = pd.concat([t2_0,t2],axis=0)
data_1 = pd.merge(data_1,t2,how='left',on='时间戳')


t1 = data.copy()
t1.columns = ['时间戳','辐照度+1','风速+1','风向+1','温度+1','压强+1','湿度+1']
t1_0 = t1[-1:].copy() # 获取最后一行，用于合并
t1['时间戳'] = t1['时间戳'].map(lambda x: x-pd.Timedelta(seconds=900))  #15分*60=900秒
t1 = pd.concat([t1,t1_0],axis=0)
data_1 = pd.merge(data_1,t1,how='left',on='时间戳')

t2 = data.copy()
t2.columns = ['时间戳','辐照度+2','风速+2','风向+2','温度+2','压强+2','湿度+2']
t2_0 = t2[-2:].copy() # 获取最后两行，用于合并
t2['时间戳'] = t2['时间戳'].map(lambda x:  x-pd.Timedelta(seconds=1800))  #30分*60=900秒
t2 = pd.concat([t2,t2_0],axis=0)
data_1 = pd.merge(data_1,t2,how='left',on='时间戳')

t3 = data.copy()
t3.columns = ['时间戳','辐照度+3','风速+3','风向+3','温度+3','压强+3','湿度+3']
t3_0 = t3[-3:].copy() # 获取最后三行，用于合并
t3['时间戳'] = t3['时间戳'].map(lambda x: x-pd.Timedelta(seconds=2700))  #45分*60=2700秒
t3 = pd.concat([t3,t3_0],axis=0)
data_1 = pd.merge(data_1,t3,how='left',on='时间戳')

In [24]:
data_1.head()

,时间,辐照度,风速,风向,温度,湿度,压强,实际辐照度,实际功率,id,时间戳,辐照度-1,风速-1,风向-1,温度-1,压强-1,湿度-1,辐照度-2,风速-2,风向-2,温度-2,压强-2,湿度-2,辐照度+1,风速+1,风向+1,温度+1,压强+1,湿度+1,辐照度+2,风速+2,风向+2,温度+2,压强+2,湿度+2,辐照度+3,风速+3,风向+3,温度+3,压强+3,湿度+3
0,2017-01-01 00:15:00,-1.0,-0.855915,0.291529,-0.459561,-0.207037,0.043513,0.0,0.039000,NaN,2017-01-01 00:15:00,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037,-1.0,-0.851950,0.395271,-0.462905,0.042722,-0.200263,-1.0,-0.847984,0.497236,-0.465831,0.041930,-0.192790,-1.0,-0.844019,0.592254,-0.469592,0.041139,-0.184763
1,2017-01-01 00:30:00,-1.0,-0.851950,0.395271,-0.462905,-0.200263,0.042722,0.0,0.039000,NaN,2017-01-01 00:30:00,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037,-1.0,-0.851950,0.395271,-0.462905,0.042722,-0.200263,-1.0,-0.847984,0.497236,-0.465831,0.041930,-0.192790,-1.0,-0.844019,0.592254,-0.469592,0.041139,-0.184763,-1.0,-0.838731,0.675103,-0.473354,0.040348,-0.176346
2,2017-01-01 00:45:00,-1.0,-0.847984,0.497236,-0.465831,-0.192790,0.041930,0.0,0.042333,NaN,2017-01-01 00:45:00,-1.0,-0.851950,0.395271,-0.462905,0.042722,-0.200263,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037,-1.0,-0.844019,0.592254,-0.469592,0.041139,-0.184763,-1.0,-0.838731,0.675103,-0.473354,0.040348,-0.176346,-1.0,-0.833443,0.740727,-0.477116,0.039557,-0.167724
3,2017-01-01 01:00:00,-1.0,-0.844019,0.592254,-0.469592,-0.184763,0.041139,0.0,0.042667,NaN,2017-01-01 01:00:00,-1.0,-0.847984,0.497236,-0.465831,0.041930,-0.192790,-1.0,-0.851950,0.395271,-0.462905,0.042722,-0.200263,-1.0,-0.838731,0.675103,-0.473354,0.040348,-0.176346,-1.0,-0.833443,0.740727,-0.477116,0.039557,-0.167724,-1.0,-0.826834,0.783847,-0.481296,0.038766,-0.159020
4,2017-01-01 01:15:00,-1.0,-0.838731,0.675103,-0.473354,-0.176346,0.040348,0.0,0.042667,NaN,2017-01-01 01:15:00,-1.0,-0.844019,0.592254,-0.469592,0.041139,-0.184763,-1.0,-0.847984,0.497236,-0.465831,0.041930,-0.192790,-1.0,-0.833443,0.740727,-0.477116,0.039557,-0.167724,-1.0,-0.826834,0.783847,-0.481296,0.038766,-0.159020,-1.0,-0.820225,0.799405,-0.485893,0.037184,-0.150397


In [27]:
data_1[data_1['辐照度+2'].isnull()].index

Int64Index([ 4407,  4408,  4918,  5868,  6146,  6147, 11890, 11891, 12073,
            12264, 12746, 12747, 12842, 13863, 13864, 14885, 14886, 15741,
            15742, 18637, 18638, 18925, 19786, 20638, 20639, 22695, 22755,
            23437, 25053, 25054, 25070, 25073, 25074, 25078, 25079, 26990,
            26991, 44270, 44271, 47092, 47093, 53044, 53045, 54143, 54144,
            54815, 54816, 57983, 57984, 62207, 62208, 62399, 62400, 62495,
            62496, 62687, 62688, 63071, 63072, 64991, 64992, 65183, 65184,
            67199, 67200, 68255, 68256, 68447, 68448, 70079, 70080, 70175,
            70176, 70953, 72214, 72407, 73293, 73561, 73818, 73819, 73828,
            73932, 74381, 74382, 75685, 75686],
           dtype='int64')

In [28]:
data_1.isnull().sum()

时间           0
辐照度          0
风速           0
风向           0
温度           0
湿度           0
压强           0
实际辐照度    12489
实际功率     12489
id       65760
时间戳          0
辐照度-1       53
风速-1        53
风向-1        53
温度-1        53
压强-1        53
湿度-1        53
辐照度-2       86
风速-2        86
风向-2        86
温度-2        86
压强-2        86
湿度-2        86
辐照度+1       53
风速+1        53
风向+1        53
温度+1        53
压强+1        53
湿度+1        53
辐照度+2       86
风速+2        86
风向+2        86
温度+2        86
压强+2        86
湿度+2        86
辐照度+3      120
风速+3       120
风向+3       120
温度+3       120
压强+3       120
湿度+3       120
dtype: int64

In [29]:
# 用前15分钟的
index_list = data_1[data_1['辐照度+2'].isnull()].index.tolist() # 缺失值
for index in index_list:
    for feat in ['辐照度','风速','风向','温度','压强','湿度']:
        data_1.loc[index,feat+'+2'] = data_1.loc[index,feat+'+1']

### Rolling

In [ ]:
data_1['rolling_sum3']=data.风速.rolling(3).mean()

### diff

In [32]:
data_1['风速_diff1']=data.风速.diff(-1)

In [33]:
data_1['风速_diff2']=data.风速.diff(-2)
data_1['风速_diff1']=data.风速.diff(-1)

In [34]:
data_1[['风速','风速_diff1','风速_diff2']].head()

,风速,风速_diff1,风速_diff2
0,-0.855915,-0.003966,-0.007931
1,-0.851950,-0.003966,-0.007931
2,-0.847984,-0.003966,-0.009253
3,-0.844019,-0.005288,-0.010575
4,-0.838731,-0.005288,-0.011897


In [35]:
pd.to_datetime(data_1['时间']).dt.year

0        2017
1        2017
2        2017
3        2017
4        2017
         ... 
78244    2019
78245    2019
78246    2019
78247    2019
78248    2019
Name: 时间, Length: 78249, dtype: int64

### 年月日小时分钟特征

In [36]:
data_1['year'] = data_1['时间戳'].dt.year
data_1['month'] = data_1['时间戳'].dt.month
data_1['day'] = data_1['时间戳'].dt.day
data_1['hour'] = data_1['时间戳'].dt.hour
data_1['minute'] = data_1['时间戳'].dt.minute

In [37]:
data_1["风向"].value_counts()

-0.456783    25
-0.458006    23
-0.470786    20
-0.447059    20
-0.484122    20
             ..
-0.171061     1
 0.003251     1
 0.133998     1
-0.015420     1
-1.000000     1
Name: 风向, Length: 27173, dtype: int64

### 归一化操作

In [38]:
data_1['风向_count']= data_1['风向'].map(data_1['风向'].value_counts()).astype(int)
data_1['风向_count'] = (data_1['风向_count']-data_1['风向_count'].min()) / (data_1['风向_count'].max()-data_1['风向_count'].min())

### 同一(年，月，天，小时)在气象各特征的一些统计(最大最小中位数)

In [40]:
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month','day','hour'])[item].agg([(item+'时最小','min')])
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day','hour'])
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month','day'])[item].agg([(item+'日最小','min')])
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day'])
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month'])[item].agg([(item+'月最小','min')])
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month'])

In [ ]:
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month','day','hour'])[item].agg({item+'时最小':'min',item+'时最大':'max',item+'时中位数':'median',item+'时平均':'mean'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day','hour'])

In [ ]:
# for item in ['辐照度','风速','温度','压强','湿度']:
#     data_1[item+'时最小']=data_1.groupby(['year','month','day','hour'])[item].transform(min)
#     data_1[item+'时最小']=data_1.groupby(['year','month','day','hour'])[item].transform(min)

以上三种方法一样的效果

In [41]:
data_1['辐照度时最小'].head()

0   -1.0
1   -1.0
2   -1.0
3   -1.0
4   -1.0
Name: 辐照度时最小, dtype: float64

In [45]:
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month','day','hour'])[item].agg({item+'时中值':'median'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day','hour'])
    
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month','day'])[item].agg([(item + "日中值","median")])
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day'])
    
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month'])[item].agg({item+'月中值':'median'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month'])

In [47]:
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month','day','hour'])[item].agg([(item+'时最大','max')])
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day','hour'])
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month','day'])[item].agg({item+'日最大':'max'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day'])
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month'])[item].agg({item+'月最大':'max'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month'])
#     data_1[item+'时最大'] = data_1[item+'时最大'] - data_1[item]

气象特征原数据与平均值的差值  data[item] - data[item + "时平均"]

In [48]:
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month','day','hour'])[item].agg('mean').reset_index().rename(columns={item:item+'时平均'})
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day','hour'])
    data_1[item+'时平均'] = data_1[item] - data_1[item+'时平均']
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month','day'])[item].agg('mean').reset_index().rename(columns={item:item+'日平均'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day'])
#     data_1[item+'日平均'] = data_1[item] - data_1[item+'日平均']
# data_1.drop(['year'],axis=1,inplace=True)
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month'])[item].agg('mean').reset_index().rename(columns={item:item+'月平均'})
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month'])
    data_1[item+'月平均'] = data_1[item] - data_1[item+'月平均']
    
data_1.drop(['year'],axis=1,inplace=True)  # 删除年特征

### 多项式特征 PolynomialFeatures

In [49]:
column_names = ['辐照度','温度','压强','湿度']
poly_transformer = PolynomialFeatures(degree=2,interaction_only=False,include_bias=False)  # 无常数项和二次项
poly_data = pd.DataFrame(poly_transformer.fit_transform(data_1[column_names]),columns=poly_transformer.get_feature_names(column_names))
poly_data.drop(column_names,axis=1,inplace=True)
data_1 = pd.concat([data_1,poly_data],axis=1,sort=False)

In [50]:
data_1.head()

,时间,辐照度,风速,风向,温度,湿度,压强,实际辐照度,实际功率,id,时间戳,辐照度-1,风速-1,风向-1,温度-1,压强-1,湿度-1,辐照度-2,风速-2,风向-2,温度-2,压强-2,湿度-2,辐照度+1,风速+1,风向+1,温度+1,压强+1,湿度+1,辐照度+2,风速+2,风向+2,温度+2,压强+2,湿度+2,辐照度+3,风速+3,风向+3,温度+3,压强+3,湿度+3,rolling_sum3,风速_diff1,风速_diff2,month,day,hour,minute,风向_count,辐照度时最小,风速时最小,温度时最小,压强时最小,湿度时最小,辐照度日最小,风速日最小,温度日最小,压强日最小,湿度日最小,辐照度月最小,风速月最小,温度月最小,压强月最小,湿度月最小,辐照度日中值,风速日中值,温度日中值,压强日中值,湿度日中值,辐照度时最大,风速时最大,温度时最大,压强时最大,湿度时最大,辐照度时平均,风速时平均,温度时平均,压强时平均,湿度时平均,辐照度月平均,风速月平均,温度月平均,压强月平均,湿度月平均,辐照度^2,辐照度 温度,辐照度 压强,辐照度 湿度,温度^2,温度 压强,温度 湿度,压强^2,压强 湿度,湿度^2
0,2017-01-01 00:15:00,-1.0,-0.855915,0.291529,-0.459561,-0.207037,0.043513,0.0,0.039000,NaN,2017-01-01 00:15:00,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037,-1.0,-0.851950,0.395271,-0.462905,0.042722,-0.200263,-1.0,-0.847984,0.497236,-0.465831,0.041930,-0.192790,-1.0,-0.844019,0.592254,-0.469592,0.041139,-0.184763,NaN,-0.003966,-0.007931,1,1,0,15,0.000000,-1.0,-0.855915,-0.465831,0.041930,-0.207037,-1.0,-0.859881,-0.581191,0.015823,-0.462401,-1.0,-0.980172,-0.868339,-0.677215,-0.864589,-1.0,-0.717118,-0.467503,0.102057,-0.144895,-1.0,-0.847984,-0.459561,0.043513,-0.19279,0.0,-3.965631e-03,0.003204,0.000791,-0.007007,-0.284204,-0.217034,0.020414,0.078017,0.137106,1.0,0.459561,-0.043513,0.207037,0.211196,-0.019997,0.095146,0.001893,-0.009009,0.042864
1,2017-01-01 00:30:00,-1.0,-0.851950,0.395271,-0.462905,-0.200263,0.042722,0.0,0.039000,NaN,2017-01-01 00:30:00,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037,-1.0,-0.851950,0.395271,-0.462905,0.042722,-0.200263,-1.0,-0.847984,0.497236,-0.465831,0.041930,-0.192790,-1.0,-0.844019,0.592254,-0.469592,0.041139,-0.184763,-1.0,-0.838731,0.675103,-0.473354,0.040348,-0.176346,NaN,-0.003966,-0.007931,1,1,0,30,0.083333,-1.0,-0.855915,-0.465831,0.041930,-0.207037,-1.0,-0.859881,-0.581191,0.015823,-0.462401,-1.0,-0.980172,-0.868339,-0.677215,-0.864589,-1.0,-0.717118,-0.467503,0.102057,-0.144895,-1.0,-0.847984,-0.459561,0.043513,-0.19279,0.0,-3.333334e-10,-0.000139,0.000000,-0.000233,-0.284204,-0.213068,0.017071,0.077226,0.143881,1.0,0.462905,-0.042722,0.200263,0.214281,-0.019776,0.092703,0.001825,-0.008556,0.040105
2,2017-01-01 00:45:00,-1.0,-0.847984,0.497236,-0.465831,-0.192790,0.041930,0.0,0.042333,NaN,2017-01-01 00:45:00,-1.0,-0.851950,0.395271,-0.462905,0.042722,-0.200263,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037,-1.0,-0.844019,0.592254,-0.469592,0.041139,-0.184763,-1.0,-0.838731,0.675103,-0.473354,0.040348,-0.176346,-1.0,-0.833443,0.740727,-0.477116,0.039557,-0.167724,-0.851950,-0.003966,-0.009253,1,1,0,45,0.166667,-1.0,-0.855915,-0.465831,0.041930,-0.207037,-1.0,-0.859881,-0.581191,0.015823,-0.462401,-1.0,-0.980172,-0.868339,-0.677215,-0.864589,-1.0,-0.717118,-0.467503,0.102057,-0.144895,-1.0,-0.847984,-0.459561,0.043513,-0.19279,0.0,3.965632e-03,-0.003065,-0.000791,0.007240,-0.284204,-0.209103,0.014145,0.076435,0.151354,1.0,0.465831,-0.041930,0.192790,0.216998,-0.019532,0.089808,0.001758,-0.008084,0.037168
3,2017-01-01 01:00:00,-1.0,-0.844019,0.592254,-0.469592,-0.184763,0.041139,0.0,0.042667,NaN,2017-01-01 01:00:00,-1.0,-0.847984,0.497236,-0.465831,0.041930,-0.192790,-1.0,-0.851950,0.395271,-0.462905,0.042722,-0.200263,-1.0,-0.838731,0.675103,-0.473354,0.040348,-0.176346,-1.0,-0.833443,0.740727,-0.477116,0.039557,-0.167724,-1.0,-0.826834,0.783847,-0.481296,0.038766,-0.159020,-0.847984,-0.005288,-0.010575,1,1,1,0,0.125000,-1.0,-0.844019,-0.481296,0.038766,-0.184763,-1.0,-0.859881,-0.581191,0.015823,-0.462401,-1.0,-0.980172,-0.868339,-0.677215,-0.864589,-1.0,-0.717118,-0.467503,0.102057,-0.144895,-1.0,-0.826834,-0.469592,0.041139,-0.15902,0.0,-8.261731e-03,0.005747,0.001187,-0.012800,-0.284204,-0.205137,0.010383,0.075644,0.159381,1.0,0.469592,-0.041139,0.184763,0.220517,-0.019319,0.086763,0.001692,-0.007601,0.034137
4,2017-01-01 01:15:00,-1.0,-0.838731,0.675103,-0.473354,-0.176346,0.040348,0.0,0.042667,NaN,2017-01-01 01:15:00,-1.0,-0.844019,0.592254,-0.469592,0.041139,-0.184763,-1.0,-0.

In [51]:
def map_hour(h):
    if h < 7:
        return 1
    elif h < 10:
        return 2
    elif h < 13:
        return 3
    elif h < 16:
        return 4
    elif h < 19:
        return 5
    else:
        return 1

In [52]:
data_1['maphour'] = data_1['hour'].map(lambda x: map_hour(x))

## 训练

In [53]:
X_train_1 = data_1[data_1['实际功率'].notnull()].drop(['时间','实际辐照度','实际功率','id','时间戳'],axis=1)
y_train_1 = data_1[data_1['实际功率'].notnull()]['实际功率']
X_test_1 = data_1[data_1['实际功率'].isnull()].drop(['时间','实际辐照度','实际功率','id','时间戳'],axis=1)

In [54]:
# X_train_1 = data_1[:len(train_1)].drop(['时间','实发辐照度','实际功率','id','时间戳'],axis=1)

# y_train_1 = data_1[:len(train_1)]['实际功率']

# X_test_1 = data_1[len(train_1):].drop(['时间','实发辐照度','实际功率','id','时间戳'],axis=1)

In [55]:
train_1_features = X_train_1.columns
train_1_features

Index(['辐照度', '风速', '风向', '温度', '湿度', '压强', '辐照度-1', '风速-1', '风向-1', '温度-1',
       '压强-1', '湿度-1', '辐照度-2', '风速-2', '风向-2', '温度-2', '压强-2', '湿度-2',
       '辐照度+1', '风速+1', '风向+1', '温度+1', '压强+1', '湿度+1', '辐照度+2', '风速+2',
       '风向+2', '温度+2', '压强+2', '湿度+2', '辐照度+3', '风速+3', '风向+3', '温度+3', '压强+3',
       '湿度+3', 'rolling_sum3', '风速_diff1', '风速_diff2', 'month', 'day', 'hour',
       'minute', '风向_count', '辐照度时最小', '风速时最小', '温度时最小', '压强时最小', '湿度时最小',
       '辐照度日最小', '风速日最小', '温度日最小', '压强日最小', '湿度日最小', '辐照度月最小', '风速月最小',
       '温度月最小', '压强月最小', '湿度月最小', '辐照度日中值', '风速日中值', '温度日中值', '压强日中值', '湿度日中值',
       '辐照度时最大', '风速时最大', '温度时最大', '压强时最大', '湿度时最大', '辐照度时平均', '风速时平均',
       '温度时平均', '压强时平均', '湿度时平均', '辐照度月平均', '风速月平均', '温度月平均', '压强月平均', '湿度月平均',
       '辐照度^2', '辐照度 温度', '辐照度 压强', '辐照度 湿度', '温度^2', '温度 压强', '温度 湿度', '压强^2',
       '压强 湿度', '湿度^2', 'maphour'],
      dtype='object')

In [56]:
prediction_1 = test_1[['id']]

X_train_1_loc = X_train_1.values
X_test_1_loc = X_test_1.values
y_train_1_loc = y_train_1.values

# SKB = SelectPercentile(f_regression,percentile=95).fit(X_train_1_loc,y_train_1_loc)
# X_train_1_loc = SKB.transform(X_train_1_loc)
# X_test_1_loc = SKB.transform(X_test_1_loc)

In [57]:
len(y_train_1_loc)

65760

In [59]:
len(y_train_1_loc[y_train_1_loc>0]) # 小于0的数量

33810

In [58]:
from sklearn.metrics import mean_absolute_error

def my_metric_1(y_true,y_pred): 
    y_true_1 = y_true[y_true>=10*0.03]
    y_pred_1 = y_pred[y_true>=10*0.03]
    score = mean_absolute_error(y_true_1,y_pred_1)/10.0
    return 'my_metric_1',score,False  # False表示score不是越大越好(is_bigger_better)

In [60]:
from sklearn.model_selection import KFold,StratifiedKFold

model_1 = lgb.LGBMRegressor('gbdt',num_leaves=51,max_depth=-1,learning_rate=0.05,n_estimators=100,max_bin=255,subsample_for_bin=200000,
                         objective='regression',min_split_gain=0,min_child_weight=0.001,min_child_samples=20,subsample=1,subsample_freq=1,
                         colsample_bytree=1, reg_alpha=0, reg_lambda=0, random_state=2018,n_jobs=-1)

# 10折交叉训练，构造五10个模型
skf = KFold(n_splits=10, shuffle=True, random_state=2018)
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf.split(X_train_1_loc,y_train_1_loc)):
    print("Fold", i)
    lgb_model = model_1.fit(X_train_1_loc[train_index], y_train_1_loc[train_index],
                          eval_names =['train','valid'],
                          eval_set=[(X_train_1_loc[train_index], y_train_1_loc[train_index]), 
                                    (X_train_1_loc[test_index], y_train_1_loc[test_index])],
                          eval_metric=my_metric_1,
                          early_stopping_rounds=50)
    baseloss.append(lgb_model.best_score_['valid']['my_metric_1'])
    loss += lgb_model.best_score_['valid']['my_metric_1']
    test_pred = lgb_model.predict(X_test_1_loc, num_iteration=lgb_model.best_iteration_)
#     print('test mean:', test_pred.mean())
    prediction_1['predict_%s' % str(i)] = test_pred  # 10次预测值
print('my_metric_1:', baseloss, loss/10)

p = prediction_1.drop(['id'],axis=1)
prediction_1['prediction']=p.mean(axis=1) # 取均值

Fold 0


F:\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	train's l2: 28.0234	train's my_metric_1: 0.550479	valid's l2: 27.7759	valid's my_metric_1: 0.549615
[2]	train's l2: 25.7103	train's my_metric_1: 0.528036	valid's l2: 25.4813	valid's my_metric_1: 0.52701
[3]	train's l2: 23.619	train's my_metric_1: 0.506966	valid's l2: 23.3997	valid's my_metric_1: 0.505641
[4]	train's l2: 21.7243	train's my_metric_1: 0.48712	valid's l2: 21.5141	valid's my_metric_1: 0.485623
[5]	train's l2: 20.02	train's my_metric_1: 0.468821	valid's l2: 19.8281	valid's my_metric_1: 0.467264
[6]	train's l2: 18.4606	train's my_metric_1: 0.451269	valid's l2: 18.2936	valid's my_metric_1: 0.449844
[7]	train's l2: 17.055	train's my_metric_1: 0.434779	valid's l2: 16.8988	valid's my_metric_1: 0.433341
[8]	train's l2: 15.777	train's my_metric_1: 0.41934	valid's l2: 15.6433	valid's my_metric_1: 0.417844
[9]	train's l2: 14.6147	train's my_metric_1: 0.404893	valid's l2: 14.5033	valid's my_metric_1: 0.403456
[10]	train's l2: 13.5685	train's my_metric_1: 0.391244	valid's l2: 13.48

[82]	train's l2: 2.09795	train's my_metric_1: 0.161407	valid's l2: 2.42426	valid's my_metric_1: 0.171909
[83]	train's l2: 2.08292	train's my_metric_1: 0.160758	valid's l2: 2.41038	valid's my_metric_1: 0.171343
[84]	train's l2: 2.0686	train's my_metric_1: 0.160181	valid's l2: 2.39798	valid's my_metric_1: 0.170906
[85]	train's l2: 2.05774	train's my_metric_1: 0.159733	valid's l2: 2.38974	valid's my_metric_1: 0.17056
[86]	train's l2: 2.04186	train's my_metric_1: 0.159093	valid's l2: 2.37402	valid's my_metric_1: 0.169948
[87]	train's l2: 2.03282	train's my_metric_1: 0.158686	valid's l2: 2.36848	valid's my_metric_1: 0.169694
[88]	train's l2: 2.0245	train's my_metric_1: 0.158301	valid's l2: 2.36399	valid's my_metric_1: 0.169478
[89]	train's l2: 2.0153	train's my_metric_1: 0.157868	valid's l2: 2.35772	valid's my_metric_1: 0.169184
[90]	train's l2: 2.00491	train's my_metric_1: 0.157462	valid's l2: 2.35017	valid's my_metric_1: 0.168886
[91]	train's l2: 1.98822	train's my_metric_1: 0.156738	vali

F:\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
F:\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


...

[1]	train's l2: 28.0002	train's my_metric_1: 0.550873	valid's l2: 28.0224	valid's my_metric_1: 0.548622
[2]	train's l2: 25.6952	train's my_metric_1: 0.528619	valid's l2: 25.6978	valid's my_metric_1: 0.52601
[3]	train's l2: 23.6065	train's my_metric_1: 0.507554	valid's l2: 23.5997	valid's my_metric_1: 0.504879
[4]	train's l2: 21.7183	train's my_metric_1: 0.487727	valid's l2: 21.7022	valid's my_metric_1: 0.484967
[5]	train's l2: 20.0143	train's my_metric_1: 0.469162	valid's l2: 19.9858	valid's my_metric_1: 0.46623
[6]	train's l2: 18.4658	train's my_metric_1: 0.451829	valid's l2: 18.4423	valid's my_metric_1: 0.448804
[7]	train's l2: 17.0672	train's my_metric_1: 0.435464	valid's l2: 17.0424	valid's my_metric_1: 0.432268
[8]	train's l2: 15.7886	train's my_metric_1: 0.419951	valid's l2: 15.7648	valid's my_metric_1: 0.416741
[9]	train's l2: 14.6264	train's my_metric_1: 0.405481	valid's l2: 14.5974	valid's my_metric_1: 0.40234
[10]	train's l2: 13.5801	train's my_metric_1: 0.391948	valid's l2: 

[85]	train's l2: 2.04672	train's my_metric_1: 0.15924	valid's l2: 2.28874	valid's my_metric_1: 0.167148
[86]	train's l2: 2.03774	train's my_metric_1: 0.158798	valid's l2: 2.28328	valid's my_metric_1: 0.166849
[87]	train's l2: 2.0278	train's my_metric_1: 0.158366	valid's l2: 2.27643	valid's my_metric_1: 0.166594
[88]	train's l2: 2.0197	train's my_metric_1: 0.157986	valid's l2: 2.27053	valid's my_metric_1: 0.16631
[89]	train's l2: 2.00796	train's my_metric_1: 0.157445	valid's l2: 2.2613	valid's my_metric_1: 0.165865
[90]	train's l2: 1.99919	train's my_metric_1: 0.157059	valid's l2: 2.2546	valid's my_metric_1: 0.165553
[91]	train's l2: 1.98907	train's my_metric_1: 0.156622	valid's l2: 2.24568	valid's my_metric_1: 0.165215
[92]	train's l2: 1.97535	train's my_metric_1: 0.156038	valid's l2: 2.23333	valid's my_metric_1: 0.164676
[93]	train's l2: 1.96584	train's my_metric_1: 0.155641	valid's l2: 2.22589	valid's my_metric_1: 0.164401
[94]	train's l2: 1.95236	train's my_metric_1: 0.155037	valid'

F:\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	train's l2: 28.016	train's my_metric_1: 0.550567	valid's l2: 27.9171	valid's my_metric_1: 0.55158
[2]	train's l2: 25.716	train's my_metric_1: 0.52821	valid's l2: 25.6169	valid's my_metric_1: 0.529088
[3]	train's l2: 23.6229	train's my_metric_1: 0.507127	valid's l2: 23.5256	valid's my_metric_1: 0.507742
[4]	train's l2: 21.7421	train's my_metric_1: 0.48744	valid's l2: 21.6482	valid's my_metric_1: 0.487939
[5]	train's l2: 20.0247	train's my_metric_1: 0.468754	valid's l2: 19.9391	valid's my_metric_1: 0.469266
[6]	train's l2: 18.4684	train's my_metric_1: 0.4514	valid's l2: 18.3951	valid's my_metric_1: 0.451834
[7]	train's l2: 17.0591	train's my_metric_1: 0.434913	valid's l2: 16.9923	valid's my_metric_1: 0.43526
[8]	train's l2: 15.7863	train's my_metric_1: 0.419554	valid's l2: 15.7342	valid's my_metric_1: 0.41991
[9]	train's l2: 14.6297	train's my_metric_1: 0.405002	valid's l2: 14.5857	valid's my_metric_1: 0.405342
[10]	train's l2: 13.5861	train's my_metric_1: 0.391731	valid's l2: 13.553

[81]	train's l2: 2.11348	train's my_metric_1: 0.16214	valid's l2: 2.3987	valid's my_metric_1: 0.170467
[82]	train's l2: 2.09738	train's my_metric_1: 0.161517	valid's l2: 2.38559	valid's my_metric_1: 0.169955
[83]	train's l2: 2.082	train's my_metric_1: 0.160832	valid's l2: 2.3717	valid's my_metric_1: 0.169358
[84]	train's l2: 2.0676	train's my_metric_1: 0.160202	valid's l2: 2.359	valid's my_metric_1: 0.168913
[85]	train's l2: 2.05746	train's my_metric_1: 0.159752	valid's l2: 2.34951	valid's my_metric_1: 0.168533
[86]	train's l2: 2.04823	train's my_metric_1: 0.159343	valid's l2: 2.344	valid's my_metric_1: 0.16823
[87]	train's l2: 2.03734	train's my_metric_1: 0.158852	valid's l2: 2.33439	valid's my_metric_1: 0.167796
[88]	train's l2: 2.02337	train's my_metric_1: 0.158231	valid's l2: 2.32143	valid's my_metric_1: 0.167154
[89]	train's l2: 2.01343	train's my_metric_1: 0.157794	valid's l2: 2.31412	valid's my_metric_1: 0.166789
[90]	train's l2: 2	train's my_metric_1: 0.157226	valid's l2: 2.303

###  特征重要性

In [62]:
train_features = X_train_1.columns
model_1.feature_importances_
pd.DataFrame({'features':train_features, 'imp':model_1.feature_importances_}).sort_values('imp',ascending=False)

,features,imp
40,day,346
63,湿度日中值,311
53,湿度日最小,293
51,温度日最小,257
59,辐照度日中值,231
...,...,...
71,温度时平均,3
43,风向_count,1
54,辐照度月最小,0
49,辐照度日最小,0


In [64]:
prediction_1.head()

,id,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9,prediction
0,1,-0.013323,-0.015494,-0.013467,-0.014923,-0.013946,-0.012038,-0.013139,-0.013413,-0.014269,-0.015479,-0.013949
1,2,-0.013323,-0.015494,-0.013467,-0.014923,-0.013946,-0.012038,-0.013139,-0.013413,-0.014269,-0.015479,-0.013949
2,3,-0.013323,-0.015494,-0.013467,-0.014923,-0.013946,-0.012038,-0.013139,-0.013413,-0.014269,-0.015479,-0.013949
3,4,-0.013323,-0.015494,-0.013467,-0.014923,-0.013946,-0.012038,-0.013139,-0.013413,-0.014269,-0.015479,-0.013949
4,5,-0.013323,-0.015494,-0.013467,-0.014923,-0.013946,-0.012038,-0.013139,-0.013413,-0.014269,-0.015479,-0.013949


In [ ]:
# 合并10个厂的预测结果

In [ ]:
prediction = pd.concat([prediction_1,prediction_2],axis=0,sort=False)
prediction = pd.concat([prediction,prediction_3],axis=0,sort=False)
prediction = pd.concat([prediction,prediction_4],axis=0,sort=False)
prediction

In [ ]:
now = datetime.datetime.now()
now = now.strftime('%m-%d-%H-%M')

prediction.rename(columns={'prediction':'prediction'},inplace=True)
prediction[['id', 'prediction']].to_csv("./result/baseline_%s.csv" % now, index=False)

### target encoder

In [65]:
data_1['压强_ctr']=data_1.groupby(['压强'])['实际功率'].transform('mean')
data_1['压强_ctr'].head()

0    4.621506
1    3.547774
2    3.450948
3    4.307555
4    4.405996
Name: 压强_ctr, dtype: float64

In [ ]:
# 适合二分类

In [ ]:
data_1['压强_ctr'] = data_1.groupby(['压强'])['label'].transform('sum')
data_1['压强_count'] = data_1.groupby(['压强'])['压强'].transform('count')
data_1['压强_ctrz'] = data_1['压强_ctr']/(data_1['压强_count']+0.0001)